# Pretraining ParallelGNN Models with solubility dataset

1. Pretraining solubility dataset using different dataset sizes and similarity levels

## Note:
1. Suggest to skip this notebook and refer to the pre-trained models which were obtained and saved in pretrained_models folder. 
2. This is because of pytorch reproducibility issue. Despite seeding everything using the seed_everything function in config.py, models trained still report slightly different MSE loss values due to the nondeterministic approach. When compared, weights were different. As such, when pre-training is repeated, there might be a chance that the MSE loss would not be the same and hence results might not be the same as well.
3. Since pre-training models provide the initial point for the training, please make use of the pre-trained models that were obtained and uploaded to GitHub for reproducibility of results obtained.
4. In the event that you will like to repeat pre-training, to keep things consistent and reproducible, please train the models such that they have the same/similar loss values as the one that was obtained

| Data type             | Pretraining Epochs | MSE Loss         |
| --------------------- | ------------------ |------------------|
| low                   | 20                 |0.9308284640312194|
| low                   | 40                 |0.7136518269777298|
| low                   | 60                 |0.5759974002838135|
| mid                   | 20                 |0.8083086103200913|
| mid                   | 40                 |0.5740450233221054|
| mid                   | 60                 |0.4558365240693092|
| high                  | 20                 |0.7509024858474731|
| high                  | 40                 |0.5216782301664352|
| high                  | 60                 |0.4045856937766075|
| mid (9844)            | 20                 |0.755623785349039 |
| mid (9844)            | 40                 |0.5991875949578408|
| mid (9984)            | 60                 |0.4397607071277423|


In [1]:
# import all required materials

import torch
import numpy as np
from torch_geometric.loader import DataLoader
from sklearn.model_selection import KFold
import os

from model import ParallelGNN
from config import NUM_FEATURES, NUM_TARGET, EDGE_DIM, DEVICE, SEED_NO, PATIENCE, EPOCHS, NUM_GRAPHS_PER_BATCH, N_SPLITS, best_params_parallel
from engine import EngineSol, EngineHOB
from utils import seed_everything, LoadHOBDataset, LoadSolDataset

# Pre-training models with 3 different epochs values 

In [2]:
def run_training_trf_learning_model(train_loader, params, pretrained_model_path, epochs):

    '''
    Define function to pretrain model with solubililty dataset 

    Args:
    train_loader: DataLoader class from pytorch geometric containing train dataset
    params (dict): Dictionary containing hyperparameters
    pretrained_model_path (str): path to save the pretrained model
    epochs (int): Number of epochs to pretrain the model

    Return:
    loss: final train loss  
    '''

    model = ParallelGNN(num_features=NUM_FEATURES, num_targets=NUM_TARGET, num_gin_layers=params['num_gin_layers'], num_graph_trans_layers=params['num_graph_trans_layers'], hidden_size=params['hidden_size'], 
                        n_heads=params['n_heads'], dropout=params['dropout'], edge_dim=EDGE_DIM)         
    model.to(DEVICE)
    optimizer=torch.optim.Adam(model.parameters(),lr = params['learning_rate'])
    eng = EngineSol(model, optimizer, device=DEVICE)


    for epoch in range(epochs):
        train_loss = eng.train(train_loader)
        print(f'Epoch: {epoch+1}/{epochs}, train loss : {train_loss}')
        print('Saving model...')
        torch.save(model.state_dict(), pretrained_model_path)

    return train_loss

### Workflow to pretrain models
1. seed_everything with a seed no of 0. SEED_NO available from the config.py file
2. load in dataset. Root path of the dataset and raw file name of the dataset is required. root path --> must contain 2 folders, raw and processed. raw csv file will be kept in the raw folder
3. first run will include the conversion of smiles to graph data --> take about 1 min to be done. Subsequent run will just load from what was already loaded previously
4. model will then be pre-trained and saved into the path listed 

### For low similarity data

In [3]:
epochs = 20
params = best_params_parallel

# Seed everything to ensure reproducibility. SEED_NO = 0 available in the config.py file 
seed_everything(SEED_NO)

# load solubility dataset. root --> root path of the data, raw filename --> name of raw file 
train_dataset_low = LoadSolDataset(root='./data/graph_data/trf_learning_logS_low/', raw_filename='logS_data_for_pretrained_model_low.csv')
train_loader_low = DataLoader(train_dataset_low, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)

# path to save the pre-trained models
pretrained_model_path = f'./trf_learning_models/pretrained_models/parallel/low/pretrained_parallel_model_{epochs}_epoch.pt'

# train the mode
train_loss = run_training_trf_learning_model(train_loader_low, params, pretrained_model_path, epochs)
print(f'train loss: {train_loss}')

Epoch: 1/20, train loss : 5.5211021900177
Saving model...
Epoch: 2/20, train loss : 3.141400396823883
Saving model...
Epoch: 3/20, train loss : 2.2217857837677
Saving model...
Epoch: 4/20, train loss : 1.924638706445694
Saving model...
Epoch: 5/20, train loss : 1.621340012550354
Saving model...
Epoch: 6/20, train loss : 1.4470352351665496
Saving model...
Epoch: 7/20, train loss : 1.502454122900963
Saving model...
Epoch: 8/20, train loss : 1.2814750909805297
Saving model...
Epoch: 9/20, train loss : 1.2383556008338927
Saving model...
Epoch: 10/20, train loss : 1.2024710565805434
Saving model...
Epoch: 11/20, train loss : 1.0959697127342225
Saving model...
Epoch: 12/20, train loss : 1.0768183469772339
Saving model...
Epoch: 13/20, train loss : 1.0273351699113846
Saving model...
Epoch: 14/20, train loss : 1.181999284029007
Saving model...
Epoch: 15/20, train loss : 1.0718557298183442
Saving model...
Epoch: 16/20, train loss : 0.9875313371419907
Saving model...
Epoch: 17/20, train loss : 1

In [4]:
epochs = 40
params = best_params_parallel

seed_everything(SEED_NO)
train_dataset_low = LoadSolDataset(root='./data/graph_data/trf_learning_logS_low/', raw_filename='logS_data_for_pretrained_model_low.csv')
train_loader_low = DataLoader(train_dataset_low, batch_size=256, shuffle=True)
pretrained_model_path = f'./trf_learning_models/pretrained_models/parallel/low/pretrained_parallel_model_{epochs}_epoch.pt'
train_loss = run_training_trf_learning_model(train_loader_low, params, pretrained_model_path, epochs)
print(f'train loss: {train_loss}')

Epoch: 1/40, train loss : 5.521102237701416
Saving model...
Epoch: 2/40, train loss : 3.141400468349457
Saving model...
Epoch: 3/40, train loss : 2.2217830300331114
Saving model...
Epoch: 4/40, train loss : 1.9249813914299012
Saving model...
Epoch: 5/40, train loss : 1.621603113412857
Saving model...
Epoch: 6/40, train loss : 1.4567142754793168
Saving model...
Epoch: 7/40, train loss : 1.5298889398574829
Saving model...
Epoch: 8/40, train loss : 1.3079859733581543
Saving model...
Epoch: 9/40, train loss : 1.205931144952774
Saving model...
Epoch: 10/40, train loss : 1.2682111114263535
Saving model...
Epoch: 11/40, train loss : 1.135237944126129
Saving model...
Epoch: 12/40, train loss : 1.1480298429727553
Saving model...
Epoch: 13/40, train loss : 1.0344093769788743
Saving model...
Epoch: 14/40, train loss : 1.2882357776165008
Saving model...
Epoch: 15/40, train loss : 1.100435796380043
Saving model...
Epoch: 16/40, train loss : 1.0175404638051986
Saving model...
Epoch: 17/40, train los

In [5]:
epochs = 60
params = best_params_parallel


seed_everything(SEED_NO)
train_dataset_low = LoadSolDataset(root='./data/graph_data/trf_learning_logS_low/', raw_filename='logS_data_for_pretrained_model_low.csv')
train_loader_low = DataLoader(train_dataset_low, batch_size=256, shuffle=True)
pretrained_model_path = f'./trf_learning_models/pretrained_models/parallel/low/pretrained_parallel_model_{epochs}_epoch.pt'
train_loss = run_training_trf_learning_model(train_loader_low, params, pretrained_model_path, epochs)
print(f'train loss: {train_loss}')

Epoch: 1/60, train loss : 5.521102249622345
Saving model...
Epoch: 2/60, train loss : 3.141400420665741
Saving model...
Epoch: 3/60, train loss : 2.2217946290969848
Saving model...
Epoch: 4/60, train loss : 1.9249893367290496
Saving model...
Epoch: 5/60, train loss : 1.6205761551856994
Saving model...
Epoch: 6/60, train loss : 1.4505530506372453
Saving model...
Epoch: 7/60, train loss : 1.4902881979942322
Saving model...
Epoch: 8/60, train loss : 1.2721269994974136
Saving model...
Epoch: 9/60, train loss : 1.2233723133802414
Saving model...
Epoch: 10/60, train loss : 1.214267760515213
Saving model...
Epoch: 11/60, train loss : 1.0968627721071242
Saving model...
Epoch: 12/60, train loss : 1.082870602607727
Saving model...
Epoch: 13/60, train loss : 1.0158810049295426
Saving model...
Epoch: 14/60, train loss : 1.1936377555131912
Saving model...
Epoch: 15/60, train loss : 1.0723388195037842
Saving model...
Epoch: 16/60, train loss : 0.9731283366680146
Saving model...
Epoch: 17/60, train l

### For mid similarity data

In [6]:
epochs = 20
params = best_params_parallel


seed_everything(SEED_NO)
train_dataset_mid = LoadSolDataset(root='./data/graph_data/trf_learning_logS_mid/', raw_filename='logS_data_for_pretrained_model_mid.csv')
train_loader_mid = DataLoader(train_dataset_mid, batch_size=256, shuffle=True)
pretrained_model_path = f'./trf_learning_models/pretrained_models/parallel/mid/pretrained_parallel_model_{epochs}_epoch.pt'

train_loss = run_training_trf_learning_model(train_loader_mid, params, pretrained_model_path, epochs)
print(f'train loss: {train_loss}')

Epoch: 1/20, train loss : 4.544011640548706
Saving model...
Epoch: 2/20, train loss : 2.318880182504654
Saving model...
Epoch: 3/20, train loss : 1.9137514233589172
Saving model...
Epoch: 4/20, train loss : 1.4855545341968537
Saving model...
Epoch: 5/20, train loss : 1.470324409008026
Saving model...
Epoch: 6/20, train loss : 1.3583060413599015
Saving model...
Epoch: 7/20, train loss : 1.328382682800293
Saving model...
Epoch: 8/20, train loss : 1.1010364681482314
Saving model...
Epoch: 9/20, train loss : 1.0257614970207214
Saving model...
Epoch: 10/20, train loss : 0.9629837214946747
Saving model...
Epoch: 11/20, train loss : 1.028915172815323
Saving model...
Epoch: 12/20, train loss : 0.8849341839551925
Saving model...
Epoch: 13/20, train loss : 0.9177361816167832
Saving model...
Epoch: 14/20, train loss : 0.8962326943874359
Saving model...
Epoch: 15/20, train loss : 0.8262790441513062
Saving model...
Epoch: 16/20, train loss : 0.8209617614746094
Saving model...
Epoch: 17/20, train lo

In [7]:
epochs = 40
params = best_params_parallel


seed_everything(SEED_NO)
train_dataset_mid = LoadSolDataset(root='./data/graph_data/trf_learning_logS_mid/', raw_filename='logS_data_for_pretrained_model_mid.csv')
train_loader_mid = DataLoader(train_dataset_mid, batch_size=256, shuffle=True)
pretrained_model_path = f'./trf_learning_models/pretrained_models/parallel/mid/pretrained_parallel_model_{epochs}_epoch.pt'

train_loss = run_training_trf_learning_model(train_loader_mid, params, pretrained_model_path, epochs)
print(f'train loss: {train_loss}')

Epoch: 1/40, train loss : 4.544017827510833
Saving model...
Epoch: 2/40, train loss : 2.3186146318912506
Saving model...
Epoch: 3/40, train loss : 1.9133813500404357
Saving model...
Epoch: 4/40, train loss : 1.4853240251541138
Saving model...
Epoch: 5/40, train loss : 1.4832194328308106
Saving model...
Epoch: 6/40, train loss : 1.4374340355396271
Saving model...
Epoch: 7/40, train loss : 1.5273986399173736
Saving model...
Epoch: 8/40, train loss : 1.1304687559604645
Saving model...
Epoch: 9/40, train loss : 1.0523628145456314
Saving model...
Epoch: 10/40, train loss : 1.0102443516254425
Saving model...
Epoch: 11/40, train loss : 0.9693474620580673
Saving model...
Epoch: 12/40, train loss : 0.8983599960803985
Saving model...
Epoch: 13/40, train loss : 0.9268206745386124
Saving model...
Epoch: 14/40, train loss : 0.8757076412439346
Saving model...
Epoch: 15/40, train loss : 0.8673748910427094
Saving model...
Epoch: 16/40, train loss : 0.816517812013626
Saving model...
Epoch: 17/40, train

In [8]:
epochs = 60
params = best_params_parallel


seed_everything(SEED_NO)
train_dataset_mid = LoadSolDataset(root='./data/graph_data/trf_learning_logS_mid/', raw_filename='logS_data_for_pretrained_model_mid.csv')
train_loader_mid = DataLoader(train_dataset_mid, batch_size=256, shuffle=True)
pretrained_model_path = f'./trf_learning_models/pretrained_models/parallel/mid/pretrained_parallel_model_{epochs}_epoch.pt'

train_loss = run_training_trf_learning_model(train_loader_mid, params, pretrained_model_path, epochs)
print(f'train loss: {train_loss}')

Epoch: 1/60, train loss : 4.544017779827118
Saving model...
Epoch: 2/60, train loss : 2.3186147034168245
Saving model...
Epoch: 3/60, train loss : 1.9133814930915833
Saving model...
Epoch: 4/60, train loss : 1.4853241264820098
Saving model...
Epoch: 5/60, train loss : 1.4832176327705384
Saving model...
Epoch: 6/60, train loss : 1.4374283611774445
Saving model...
Epoch: 7/60, train loss : 1.5273414522409439
Saving model...
Epoch: 8/60, train loss : 1.130295193195343
Saving model...
Epoch: 9/60, train loss : 1.0499890804290772
Saving model...
Epoch: 10/60, train loss : 1.0120443105697632
Saving model...
Epoch: 11/60, train loss : 0.9729029893875122
Saving model...
Epoch: 12/60, train loss : 0.8972269594669342
Saving model...
Epoch: 13/60, train loss : 0.9287420600652695
Saving model...
Epoch: 14/60, train loss : 0.8851360738277435
Saving model...
Epoch: 15/60, train loss : 0.8213381260633469
Saving model...
Epoch: 16/60, train loss : 0.8382677048444748
Saving model...
Epoch: 17/60, train

### For high similarity data

In [9]:
epochs = 20
params = best_params_parallel

seed_everything(SEED_NO)
train_dataset_high = LoadSolDataset(root='./data/graph_data/trf_learning_logS_high/', raw_filename='logS_data_for_pretrained_model_high.csv')
train_loader_high = DataLoader(train_dataset_high, batch_size=256, shuffle=True)
pretrained_model_path = f'./trf_learning_models/pretrained_models/parallel/high/pretrained_parallel_model_{epochs}_epoch.pt'

train_loss = run_training_trf_learning_model(train_loader_high, params, pretrained_model_path, epochs)
print(f'train loss: {train_loss}')

Epoch: 1/20, train loss : 4.642562830448151
Saving model...
Epoch: 2/20, train loss : 2.101711893081665
Saving model...
Epoch: 3/20, train loss : 1.7573925614356996
Saving model...
Epoch: 4/20, train loss : 1.3711799442768098
Saving model...
Epoch: 5/20, train loss : 1.2677027255296707
Saving model...
Epoch: 6/20, train loss : 1.1138039022684096
Saving model...
Epoch: 7/20, train loss : 1.124337151646614
Saving model...
Epoch: 8/20, train loss : 0.9639853119850159
Saving model...
Epoch: 9/20, train loss : 0.9671495497226715
Saving model...
Epoch: 10/20, train loss : 0.9340693712234497
Saving model...
Epoch: 11/20, train loss : 0.8093831866979599
Saving model...
Epoch: 12/20, train loss : 0.8000556081533432
Saving model...
Epoch: 13/20, train loss : 0.7660887330770493
Saving model...
Epoch: 14/20, train loss : 0.7237061128020287
Saving model...
Epoch: 15/20, train loss : 0.720808693766594
Saving model...
Epoch: 16/20, train loss : 0.7477890074253082
Saving model...
Epoch: 17/20, train l

In [10]:
epochs = 40
params = best_params_parallel


seed_everything(SEED_NO)
train_dataset_high = LoadSolDataset(root='./data/graph_data/trf_learning_logS_high/', raw_filename='logS_data_for_pretrained_model_high.csv')
train_loader_high = DataLoader(train_dataset_high, batch_size=256, shuffle=True)
pretrained_model_path = f'./trf_learning_models/pretrained_models/parallel/high/pretrained_parallel_model_{epochs}_epoch.pt'

train_loss = run_training_trf_learning_model(train_loader_high, params, pretrained_model_path, epochs)
print(f'train loss: {train_loss}')

Epoch: 1/40, train loss : 4.642562794685364
Saving model...
Epoch: 2/40, train loss : 2.1017646968364714
Saving model...
Epoch: 3/40, train loss : 1.757392978668213
Saving model...
Epoch: 4/40, train loss : 1.3698823034763337
Saving model...
Epoch: 5/40, train loss : 1.2634358733892441
Saving model...
Epoch: 6/40, train loss : 1.1285671651363374
Saving model...
Epoch: 7/40, train loss : 1.1352617740631104
Saving model...
Epoch: 8/40, train loss : 0.9667869120836258
Saving model...
Epoch: 9/40, train loss : 0.974018257856369
Saving model...
Epoch: 10/40, train loss : 0.9377013027668
Saving model...
Epoch: 11/40, train loss : 0.8118010640144349
Saving model...
Epoch: 12/40, train loss : 0.8056966811418533
Saving model...
Epoch: 13/40, train loss : 0.7730775952339173
Saving model...
Epoch: 14/40, train loss : 0.7298048958182335
Saving model...
Epoch: 15/40, train loss : 0.7278229787945747
Saving model...
Epoch: 16/40, train loss : 0.7500758439302444
Saving model...
Epoch: 17/40, train los

In [11]:
epochs = 60
params = best_params_parallel


seed_everything(SEED_NO)
train_dataset_high = LoadSolDataset(root='./data/graph_data/trf_learning_logS_high/', raw_filename='logS_data_for_pretrained_model_high.csv')
train_loader_high = DataLoader(train_dataset_high, batch_size=256, shuffle=True)
pretrained_model_path = f'./trf_learning_models/pretrained_models/parallel/high/pretrained_parallel_model_{epochs}_epoch.pt'

train_loss = run_training_trf_learning_model(train_loader_high, params, pretrained_model_path, epochs)
print(f'train loss: {train_loss}')

Epoch: 1/60, train loss : 4.642562878131867
Saving model...
Epoch: 2/60, train loss : 2.1017118632793426
Saving model...
Epoch: 3/60, train loss : 1.757395625114441
Saving model...
Epoch: 4/60, train loss : 1.3700670182704926
Saving model...
Epoch: 5/60, train loss : 1.2635839074850082
Saving model...
Epoch: 6/60, train loss : 1.1241948574781417
Saving model...
Epoch: 7/60, train loss : 1.1259318739175797
Saving model...
Epoch: 8/60, train loss : 0.9613223910331726
Saving model...
Epoch: 9/60, train loss : 0.9676713794469833
Saving model...
Epoch: 10/60, train loss : 0.9594990938901902
Saving model...
Epoch: 11/60, train loss : 0.8161195397377015
Saving model...
Epoch: 12/60, train loss : 0.8084379732608795
Saving model...
Epoch: 13/60, train loss : 0.7736717820167541
Saving model...
Epoch: 14/60, train loss : 0.7311766460537911
Saving model...
Epoch: 15/60, train loss : 0.7311332583427429
Saving model...
Epoch: 16/60, train loss : 0.7419393539428711
Saving model...
Epoch: 17/60, train

### For mid similarity dataset of 9844 molecules

In [12]:
epochs = 20
params = best_params_parallel

seed_everything(SEED_NO)
train_dataset_mid = LoadSolDataset(root='./data/graph_data/trf_learning_logS_mid_10x/', raw_filename='logS_data_for_pretrained_model_mid_10x.csv')
train_loader_mid = DataLoader(train_dataset_mid, batch_size=256, shuffle=True)
pretrained_model_path = f'./trf_learning_models/pretrained_models/parallel/mid_10x/pretrained_parallel_model_{epochs}_epoch.pt'

train_loss = run_training_trf_learning_model(train_loader_mid, params, pretrained_model_path, epochs)
print(f'train loss: {train_loss}')

Epoch: 1/20, train loss : 4.002060560079721
Saving model...
Epoch: 2/20, train loss : 1.8875243541521904
Saving model...
Epoch: 3/20, train loss : 1.5322926869759192
Saving model...
Epoch: 4/20, train loss : 1.2759567407461314
Saving model...
Epoch: 5/20, train loss : 1.1960181410496051
Saving model...
Epoch: 6/20, train loss : 1.1422757231272185
Saving model...
Epoch: 7/20, train loss : 1.1556563973426819
Saving model...
Epoch: 8/20, train loss : 1.0475460779972565
Saving model...
Epoch: 9/20, train loss : 0.971838096777598
Saving model...
Epoch: 10/20, train loss : 0.9097124399282993
Saving model...
Epoch: 11/20, train loss : 0.9119785932394174
Saving model...
Epoch: 12/20, train loss : 0.8857069856081253
Saving model...
Epoch: 13/20, train loss : 0.8691081985449179
Saving model...
Epoch: 14/20, train loss : 0.8624108678255326
Saving model...
Epoch: 15/20, train loss : 0.8587329143132919
Saving model...
Epoch: 16/20, train loss : 0.8370941846798627
Saving model...
Epoch: 17/20, train

In [13]:
epochs = 40
params = best_params_parallel


seed_everything(SEED_NO)
train_dataset_mid = LoadSolDataset(root='./data/graph_data/trf_learning_logS_mid_10x/', raw_filename='logS_data_for_pretrained_model_mid_10x.csv')
train_loader_mid = DataLoader(train_dataset_mid, batch_size=256, shuffle=True)
pretrained_model_path = f'./trf_learning_models/pretrained_models/parallel/mid_10x/pretrained_parallel_model_{epochs}_epoch.pt'

train_loss = run_training_trf_learning_model(train_loader_mid, params, pretrained_model_path, epochs)
print(f'train loss: {train_loss}')

Epoch: 1/40, train loss : 4.002092294203929
Saving model...
Epoch: 2/40, train loss : 1.8872488400874994
Saving model...
Epoch: 3/40, train loss : 1.5029107760160396
Saving model...
Epoch: 4/40, train loss : 1.267299500795511
Saving model...
Epoch: 5/40, train loss : 1.1722230651439765
Saving model...
Epoch: 6/40, train loss : 1.1312551956910353
Saving model...
Epoch: 7/40, train loss : 1.0996684554295661
Saving model...
Epoch: 8/40, train loss : 1.0234598227036305
Saving model...
Epoch: 9/40, train loss : 0.9531340033580096
Saving model...
Epoch: 10/40, train loss : 0.9272402708347027
Saving model...
Epoch: 11/40, train loss : 0.9092136957706549
Saving model...
Epoch: 12/40, train loss : 0.9016431692319039
Saving model...
Epoch: 13/40, train loss : 0.8663428548054818
Saving model...
Epoch: 14/40, train loss : 0.8376778685129606
Saving model...
Epoch: 15/40, train loss : 0.842821403955802
Saving model...
Epoch: 16/40, train loss : 0.8413187158413422
Saving model...
Epoch: 17/40, train 

In [14]:
epochs = 60
params = best_params_parallel


seed_everything(SEED_NO)
train_dataset_mid = LoadSolDataset(root='./data/graph_data/trf_learning_logS_mid_10x/', raw_filename='logS_data_for_pretrained_model_mid_10x.csv')
train_loader_mid = DataLoader(train_dataset_mid, batch_size=256, shuffle=True)
pretrained_model_path = f'./trf_learning_models/pretrained_models/parallel/mid_10x/pretrained_parallel_model_{epochs}_epoch.pt'

train_loss = run_training_trf_learning_model(train_loader_mid, params, pretrained_model_path, epochs)
print(f'train loss: {train_loss}')

Epoch: 1/60, train loss : 4.002092129144913
Saving model...
Epoch: 2/60, train loss : 1.8852795607004411
Saving model...
Epoch: 3/60, train loss : 1.5093652590727196
Saving model...
Epoch: 4/60, train loss : 1.2645879005774474
Saving model...
Epoch: 5/60, train loss : 1.1910986640514472
Saving model...
Epoch: 6/60, train loss : 1.111995279788971
Saving model...
Epoch: 7/60, train loss : 1.014367821889046
Saving model...
Epoch: 8/60, train loss : 0.9900711392744993
Saving model...
Epoch: 9/60, train loss : 0.9730033324314997
Saving model...
Epoch: 10/60, train loss : 0.9239229605748103
Saving model...
Epoch: 11/60, train loss : 0.9563158123921125
Saving model...
Epoch: 12/60, train loss : 0.8749214685880221
Saving model...
Epoch: 13/60, train loss : 0.8673534859449435
Saving model...
Epoch: 14/60, train loss : 0.8366829875187997
Saving model...
Epoch: 15/60, train loss : 0.832387795815101
Saving model...
Epoch: 16/60, train loss : 0.8526300207162515
Saving model...
Epoch: 17/60, train l